In [10]:
 # imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [11]:
 # Read the Mapbox API key
load_dotenv("envi.env")
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

#  Import Data

In [12]:
 # Read the census data into a Pandas DataFrame
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(Path("Data/sfo_neighborhoods_census_data.csv"), header=0)
# sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()


# Import the necessary CSVs to Pandas DataFrames
file_path1 = Path("Data/neighborhoods_coordinates.csv")
sfo_loc_data = pd.read_csv(file_path1, header=0)
# sfo_data = pd.read_csv(file_path, index_col="year")
sfo_loc_data.head()

,Neighborhood,Lat,Lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050


In [13]:
# Calculate the mean number of housing units per year (hint: use groupby) 
# housing_units_per_year
sfo_data1 = sfo_data.reset_index()
num_of_housing_unit = sfo_data1[["housing_units", "year"]]
avg_unit_per_year = num_of_housing_unit.groupby("year").sum().reset_index()
avg_unit_per_year


## Average Housing Costs in San Francisco Per Year
# Calculating the average sale price per square foot and average gross rent
avg_cost =sfo_data.groupby("year").mean().reset_index()
avg_cost =avg_cost[["year","sale_price_sqr_foot","gross_rent"]]
avg_cost

#Average Sales Price Per Year."""
sale_price_sqr_foot = avg_cost[["year","sale_price_sqr_foot"]]

#Average Gross Rent in San Francisco Per Year."""
gross_rent = avg_cost[["year","gross_rent"]]
gross_rent

## Average Prices by Neighborhood
# Creating a dataframe to for the mean value
avg_by_neigh = sfo_data.groupby(["year","neighborhood"]).mean().reset_index()
avg_by_neigh .head(10)


#  The Top 10 Most Expensive Neighborhoods

sfo_data2 = sfo_data[['neighborhood','sale_price_sqr_foot','housing_units','gross_rent']]
avg_sfo_by_neigbhoor = sfo_data2.groupby(["neighborhood"]).mean().reset_index()
avg_sfo_by_neigbhoor.head(10)
Ten_most_exp=avg_sfo_by_neigbhoor.nlargest(10,['sale_price_sqr_foot']).reset_index()
Ten_most_exp

#  Comparing cost to purchase versus rental income
 # Fetch the previously generated DataFrame that was grouped by year and neighborhood 
avg_sfo_by_year1 = sfo_data.groupby(["year","neighborhood"]).mean().reset_index()

# Neighborhood Map
# Calculate the mean values for each neighborhood

sfo_data1= sfo_data[["neighborhood","sale_price_sqr_foot","housing_units","gross_rent"]]
avg_per_neigh =sfo_data1.groupby(["neighborhood"]).mean().reset_index()
avg_per_neigh.head(10)

# Concatenate data - average values with the neighborhood locations
sfo_data_with_loc = (
    pd.concat([sfo_loc_data, avg_per_neigh], axis=1))
sfo_data_with_loc.head(3)

#sfo_data_with_loc.set_index(sfo_data_with_loc['Neighborhood'], inplace=True)

sfo_data_with_loc =sfo_data_with_loc[["Neighborhood","Lat","Lon","sale_price_sqr_foot","housing_units",
                                      "gross_rent"]]

# Rename columns
sfo_data_with_loc.columns = ["Neighborhood","Latitude","Longitude","sale_price_sqr_foot","housing_units","gross_rent"]
sfo_data_with_loc.head(10)

## Cost Analysis - Optional Challenge
### Creating a DataFrame showing the most expensive neighborhoods in San Francisco by year
# Fetch the data from all expensive neighborhoods per year.
df_expensive_neighborhoods_per_year = sfo_data[sfo_data["neighborhood"].isin(Ten_most_exp["neighborhood"])]
df_expensive_neighborhoods_per_year.head(10)

### Create a sunburst chart to conduct a costs analysis of most expensive neighborhoods in San Francisco per year
df_expensive_neighborhoods_per_year
df_expensive_neighborhoods_per_year = sfo_data[sfo_data["neighborhood"].isin(Ten_most_exp["neighborhood"])]

#sfo_data_with_loc.head(3)
sfo_data_with_loc.head()
#avg_per_neigh.head(10)

,Neighborhood,Latitude,Longitude,sale_price_sqr_foot,housing_units,gross_rent
0,Alamo Square,37.791012,-122.402100,366.020712,378401.0,2817.285714
1,Anza Vista,37.779598,-122.443451,373.382198,379050.0,3031.833333
2,Bayview,37.734670,-122.401060,204.588623,376454.0,2318.400000
3,Bayview Heights,37.728740,-122.410980,590.792839,382295.0,3739.000000
4,Bernal Heights,37.728630,-122.443050,576.746488,379374.5,3080.333333


## Panel Visualizations


In [14]:
def housing_units_per_year():
    bar_plot = px.bar(avg_unit_per_year, x="year", y="housing_units", height =400,
                          title="Hosing Unit in San Fransico from 2010 to 2016")
    return  bar_plot

# Defining function for  average gsale price per sqr foot per year  to plot the bar chart    
def average_gross_rent():
    sale_price_sqr_foot = avg_cost[["year","sale_price_sqr_foot"]]
    
    bar_plot1 = sale_price_sqr_foot.hvplot(x='year', y='sale_price_sqr_foot',height =400,
                           title ='AVERAGE PRICE PER SQRFT PER YEAR')
    return  bar_plot1

 
# Defining function for the Average Gross Rent in San Francisco Per Year to plot the bar char."""
def average_sales_price():
    bar_plot2 = gross_rent.hvplot(x='year', y='gross_rent',height =400,title ='AVERAGE GROSS RENT PER YEAR')
    return  bar_plot2

# defining function for the average price by neighborhood 
def average_price_by_neighborhood():
    bar_plot3 =avg_by_neigh.hvplot(x='year', y='sale_price_sqr_foot',groupby ='neighborhood') 
    return  bar_plot3

def average_price_by_neighborhood2():
    bar_plot3b =avg_by_neigh.hvplot(x='year', y='gross_rent',groupby ='neighborhood')
    return  bar_plot3b

# defining a function for the 10 most expensive neighborhood
def top_most_expensive_neighborhoods(): 
    bar_plot4 =Ten_most_exp.hvplot.bar(x='neighborhood',y='sale_price_sqr_foot' ,rot =90 ,height =400
                       ,title ='Top 10 Most Expensive Neigbhoods in SFO')
    return  bar_plot4

#   defining a function for the Comparison of Rent and Sales Prices of Most Expensive Neighborhoods.""" 
def most_expensive_neighborhoods_rent_sales():     
    bar_plot5 = avg_sfo_by_year1.set_index('year').hvplot(kind ='bar',stacked =False,width =700,height =400,
                groupby ='neighborhood',x='housing_units',rot =90)
    return  bar_plot5

# definiing a function for the parllel coordinates    
def parallel_coordinates():    
    bar_plot6=px.parallel_coordinates(df_expensive_neighborhoods_per_year, color='sale_price_sqr_foot',
                       title ="Parallel Coordinates Analysis of Most Expensive San Fransico Neighborhood")
    return  bar_plot6

# definiing a function for the parllel categories 
def parallel_categories():   
    bar_plot7 =px.parallel_categories(Ten_most_exp, 
                       dimensions=["neighborhood","sale_price_sqr_foot", "housing_units","gross_rent"],
                       title ="Parallel Categories Analysis of Most Expensive San Fransico Neighborhood",
                       color='sale_price_sqr_foot')
    return  bar_plot7


# defining a function for the map
def neighborhood_map():
#        # Rename columns to be 'num_sales' and 'num_foreclosures and plot the graph
    map_1 = px.scatter_mapbox(
            sfo_data_with_loc,
            lat="Latitude",
            lon="Longitude",
            color="gross_rent",
            title="Average Sale Price Per Square Foot and Gross Rent In San Francisco",
           size= "sale_price_sqr_foot",
            color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,
        width=1000
                )
    return  map_1

# Defining a function for the sunburst chart    
def sunburst():
    fig =px.sunburst(df_expensive_neighborhoods_per_year,path=['year','neighborhood'],
                 color ='gross_rent',values ='sale_price_sqr_foot',
                 width =800,height =600,
                  color_continuous_scale='Rdbu',
                  title ="Cost Analysis of the Most Expensive Neighborhood in San Francisco")
    #fig.show()
    return  fig

In [15]:
neighborhood_map()

In [16]:
# Put parallel plots in a single row
exp_neigh_plot = pn.Row(top_most_expensive_neighborhoods(),most_expensive_neighborhoods_rent_sales())

housing_unit_plot = pn.Row(housing_units_per_year(),average_sales_price, average_gross_rent()
)

neigh_analysis = pn.Column(
    "#Average Price By Neighborhood Dashboard", average_price_by_neighborhood(),average_price_by_neighborhood2(),
    most_expensive_neighborhoods_rent_sales(),top_most_expensive_neighborhoods()
)

cor_and_cat_plot = pn.Row(parallel_coordinates(),parallel_categories())
sunburst_plot = pn.Row(sunburst())

welcome_plot = pn.Column(
    " San Francisco Map ", neighborhood_map()
)



In [17]:
# Create tabs for visualizations
tabs = pn.Tabs(
    ("Welcome ",welcome_plot),
   ("Yearly Market Analysis", housing_unit_plot),
   ("Neighborhood Anaylsis",neigh_analysis),
    ("Parallel Plot Analysis",cor_and_cat_plot),
    ("Sunburst Plot Analysis",sunburst_plot),
    
)

tabs.show()

Launching server at http://localhost:55047


In [9]:
# Execute Panel dashboard using servable function.servable()
tabs.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Row
        [0] Plotly(Figure)
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive01653')
        [2] HoloViews(Curve)
    [2] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [3] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [4] HoloViews(Bars)
    [3] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)
    [4] Row
        [0] Plotly(Figure)